# Imports

In [ ]:
import pandas as pd
from datetime import timedelta
from mikeio1d import Res1D
from mikeio1d.query import QueryDataReach
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import qualitative
from pathlib import Path
from hbt_tools import utils_hbt as uh

# Functions

In [ ]:
def read_messstelle(file_path, columns):
    date_format = '%d.%m.%Y %H:%M:%S'
    usecols = ['Datum', 'Uhrzeit', *columns]
    print(usecols)
    data = pd.read_csv(file_path, sep=';', encoding='UTF-8', usecols=usecols,
        parse_dates=[[0,1]], date_format=date_format, decimal=',')
    data.set_index('Datum_Uhrzeit', inplace=True)
    data = data.astype(float, copy=True)

    return data
#-----------------------------------------------------------------------------------------------------------------------

def read_regen(file_path):
    date_format = '%d.%m.%Y %H:%M:%S'
    data = pd.read_csv(file_path, sep=';', encoding='UTF-8', usecols=['Datum', 'Uhrzeit', 'Niederschlag [mm]'],
            parse_dates=[[0,1]], date_format=date_format, decimal=',')
    data.set_index('Datum_Uhrzeit', inplace=True)

    return data
#-----------------------------------------------------------------------------------------------------------------------

def filter_by_time(ts, start_time, end_time):
    mask = (ts.index >= start_time) & (ts.index <= end_time)
    ts = ts[mask]

    return ts
#-----------------------------------------------------------------------------------------------------------------------

def get_dt(ts):
    ts = ts.copy(deep=True)
    delta_t = pd.Series(ts.index) - pd.Series(ts.index).shift()
    delta_t.iloc[0] = pd.Timedelta(seconds=0)

    return delta_t

# Parameters

In [ ]:
path_mes = Path(r"\\hunzikerwater.ch\DFSHBT\Daten\2 Projekte\1000-\1300-\1346\1346.33 V-GEP\05 Berechnungen Grundlagen"
    r"\Messkampagne\Rohdaten Messkampagne")
save_dir = path_mes / 'Plots'

links_lvl = {
    '493756': 289.25,
    '389626': 303.377,
    '159169': 335.573,
    '99210': 430.45,
}

info_messstellen = {
    'Messstelle_MO_A15': {
        'col_Q': 'Q Input [l/s]',
        'col_h': 'H Input [m]',
    },

    'Messstelle_MO_D1A': {
        'col_Q': 'Q Input [l/s]',
        'col_h': 'H Input [m]',
    },

    'Messstelle_WE_83A': {
        'col_Q': 'Q Input [l/s]',
        'col_h': 'H Input [m]',
    },

    'Messstelle_ZE_204': {
        'col_Q': 'Q Input [l/s]',
        'col_h': 'H Input [m]',
    },
}

events = {
    '30.07.2021-Möhlin': {
        'path': Path(r"H:\2 Projekte\1000-\1300-\1346\1346.33 V-GEP\05 Berechnungen Grundlagen\TP Entwässerungskonzept"
            r"\MP\04_Modell\Resultate\240207-IST-Val-Moe-30.07\Moehlin-IST-V1_Val-Moe-30.07.BaseDefault_Network_HD.res1d"),
        'rain_station': 'Möhlin',
        'mes_links': [
            ('MO_A15', '493756'),
            ('MO_D1A', '389626'),
            ('ZE_204', '159169')
        ]
    },

    '26.09.2021-Möhlin': {
        'path': Path(r"H:\2 Projekte\1000-\1300-\1346\1346.33 V-GEP\05 Berechnungen Grundlagen\TP Entwässerungskonzept"
            r"\MP\04_Modell\Resultate\240207-IST-Val-Moe-26.09\Moehlin-IST-V1_Val-Moe-26.09.BaseDefault_Network_HD.res1d"),
        'rain_station': 'Möhlin',
        'mes_links': [
            ('MO_A15', '493756'),
            ('MO_D1A', '389626'),
            ('ZE_204', '159169')
        ]
    },

    '26.09.2021-Wegenstetten': {
        'path': Path(r"H:\2 Projekte\1000-\1300-\1346\1346.33 V-GEP\05 Berechnungen Grundlagen\TP Entwässerungskonzept"
            r"\MP\04_Modell\Resultate\240207-IST-Val-Weg-26.09\Moehlin-IST-V1_Val-Weg-26.09.BaseDefault_Network_HD.res1d"),
        'rain_station': 'Wegenstetten',
        'mes_links': [
            ('WE_83A', '99210')
        ]
    },

    '10.09.2021-Wegenstetten': {
        'path': Path(r"H:\2 Projekte\1000-\1300-\1346\1346.33 V-GEP\05 Berechnungen Grundlagen\TP Entwässerungskonzept"
            r"\MP\04_Modell\Resultate\240207-IST-Val-Weg-10.09\Moehlin-IST-V1_Val-Weg-10.09.BaseDefault_Network_HD.res1d"),
        'rain_station': 'Wegenstetten',
        'mes_links': [
            ('WE_83A', '99210')
        ]
    },

    '03.08.2021-Möhlin': {
        'path': Path(r"H:\2 Projekte\1000-\1300-\1346\1346.33 V-GEP\05 Berechnungen Grundlagen\TP Entwässerungskonzept"
            r"\MP\04_Modell\Resultate\240219-IST-Val-Moe-03.08\Moehlin-IST-V1_Val-Moe-03.08.BaseDefault_Network_HD.res1d "),
        'rain_station': 'Möhlin',
        'mes_links': [
            ('MO_A15', '493756'),
            ('MO_D1A', '389626'),
            ('ZE_204', '159169')
        ]
    },

    '10.09.2021-Möhlin': {
        'path': Path(r"H:\2 Projekte\1000-\1300-\1346\1346.33 V-GEP\05 Berechnungen Grundlagen\TP Entwässerungskonzept"
            r"\MP\04_Modell\Resultate\240219-IST-Val-Moe-10.09\Moehlin-IST-V1_Val-Moe-10.09.BaseDefault_Network_HD.res1d "),
        'rain_station': 'Möhlin',
        'mes_links': [
            ('MO_A15', '493756'),
            ('MO_D1A', '389626'),
            ('ZE_204', '159169')
        ]
    },

    '16.09.2021-Wegenstetten': {
        'path': Path(r"H:\2 Projekte\1000-\1300-\1346\1346.33 V-GEP\05 Berechnungen Grundlagen\TP Entwässerungskonzept"
            r"\MP\04_Modell\Resultate\240219-IST-Val-Weg-16.09\Moehlin-IST-V1_Val-Weg-16.09.BaseDefault_Network_HD.res1d "),
        'rain_station': 'Wegenstetten',
        'mes_links': [
            ('WE_83A', '99210')
        ]
    }
}

# Code

In [ ]:
# Read data Messstellen
data_messstellen = {}
for name, cols in info_messstellen.items():
    print(name)
    path = path_mes / name
    for file in path.iterdir():
        if file.suffix == '.csv':
            data = read_messstelle(file, list(cols.values()))
            data_messstellen.update({name: data})

# Correct data WE_83A -> move 1h behind
data_messstellen['Messstelle_WE_83A'].index -= timedelta(hours=1)

In [ ]:
# Read Regendaten Möhlin
path_regen = Path(r"H:\2 Projekte\1000-\1300-\1346\1346.33 V-GEP\05 Berechnungen Grundlagen\Messkampagne"
    r"\Rohdaten Messkampagne\Regenmesser_Möhlin"
    r"\History_Data_of_Regenmesser Möhlin_from_2021-07-22 23-59-59 GMT+0200_to_2021-12-02 23-59-59 GMT+0100.csv")

rain_moh = read_regen(path_regen)

In [ ]:
# Read Regendaten Wegenstetten
path_regen = Path(r"H:\2 Projekte\1000-\1300-\1346\1346.33 V-GEP\05 Berechnungen Grundlagen\Messkampagne"
    r"\Rohdaten Messkampagne\Regenmesser_Wegenstetten"
    r"\History_Data_of_Regenmesser Wegenstetten_from_2021-07-22 23-59-59 GMT+0200_to_2021-12-02 23-59-59 GMT+0100.csv")

rain_weg = read_regen(path_regen)

In [ ]:
# Read Regendaten Meteoschweiz
id = 1903
date_format = '%d.%m.%Y'
date_from = '29.07.2021'
date_to = '02.12.2021'
rain_ms = uh.mdb_getdata(id=id, date_from=date_from, date_to=date_to, date_format=date_format)

## Plot Messstellen

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
for name, data in data_messstellen.items():
    fig.add_trace(go.Scatter(x=data.index, y=data.iloc[:,0], name=name), secondary_y=False)

fig.add_trace(go.Scatter(x=rain_moh.index, y=rain_moh.iloc[:,0], name='Regen Möhlin'), secondary_y=True)
fig.add_trace(go.Scatter(x=rain_weg.index, y=rain_weg.iloc[:,0], name='Regen Wegenstetten'), secondary_y=True)

fig.update_layout(title=f"Alle Messstellen")
fig.update_xaxes(title='Datum')
fig.update_yaxes(title='Durchfluss [l/s]', secondary_y=False)
fig.update_yaxes(title='Regen [mm]', secondary_y=True)

save_path = save_dir / f"Plot_Alle-Messstellen.html"
fig.write_html(save_path)

## Plots Validierung

In [ ]:
if not save_dir.exists():
    save_dir.mkdir()

colors_Q = qualitative.Dark2
colors_H = qualitative.Pastel2
for datum_station, params in events.items():
    file_path = params['path']
    res1d = Res1D(str(file_path))
    time_start = res1d.start_time
    time_end = res1d.end_time

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    for i, pair in enumerate(params['mes_links']):
        name_mes = f'Messstelle_{pair[0]}'
        data_mes = data_messstellen[name_mes]
        data_mes = filter_by_time(data_mes, time_start, time_end)
        name_link = pair[1]

        fig.add_trace(go.Scatter(x=data_mes.index, y=data_mes.loc[:,'Q Input [l/s]'], name=f"Durchfluss - {name_mes}",
            line=dict(color=colors_Q[i])), secondary_y=False)

        query = QueryDataReach('Discharge', name_link, chainage=0)
        data_mike = res1d.read(query)
        data_mike = data_mike * 1e3 # m3/s -> l/s
        fig.add_trace(go.Scatter(x=data_mike.index, y=data_mike.iloc[:,0], name=f"Durchfluss - {name_link}",
            line=dict(color=colors_Q[i], dash='dash')), secondary_y=False)
#-----------------------------------------------------------------------------------------------------------------------

        fig.add_trace(go.Scatter(x=data_mes.index, y=data_mes.loc[:,'H Input [m]'] * 100, name=f"Wasserstand - {name_mes}",
            line=dict(color=colors_H[i])), secondary_y=True)

        query = QueryDataReach('WaterLevel', name_link, chainage=0)
        data_mike = res1d.read(query)
        data_mike = (data_mike - links_lvl[name_link]) * 100 # m -> cm
        fig.add_trace(go.Scatter(x=data_mike.index, y=data_mike.iloc[:,0], name=f"Wasserstand - {name_link}",
            line=dict(color=colors_H[i], dash='dash')), secondary_y=True)
#-----------------------------------------------------------------------------------------------------------------------

        fig.update_layout(title=f"Modellvalidierung: {datum_station}")
        fig.update_xaxes(title='Datum')
        fig.update_yaxes(title='Durchfluss [l/s]', secondary_y=False)
        fig.update_yaxes(title='Wasserstand [cm]', secondary_y=True)

        save_path = save_dir / f"Plot-{datum_station}_Wasserstand.html"
        fig.write_html(save_path)